## 1. Setup and Configuration

In [1]:
import os
import time
import json
import requests
from typing import Dict

# Modern data stack
import psycopg
import polars as pl
import plotly.express as px
import plotly.graph_objects as go

# Configuration - Auto-detect environment
# Set NOETL_ENV=kubernetes to run against in-cluster services
# Set NOETL_ENV=localhost (default) to run against port-forwarded services
ENVIRONMENT = os.getenv("NOETL_ENV", "localhost").lower()

if ENVIRONMENT == "kubernetes":
    # In-cluster configuration
    DB_CONFIG = {
        "host": "postgres.postgres.svc.cluster.local",
        "port": "5432",
        "user": os.getenv("POSTGRES_USER", "demo"),
        "password": os.getenv("POSTGRES_PASSWORD", "demo"),
        "dbname": os.getenv("POSTGRES_DB", "demo_noetl")
    }
    NOETL_SERVER_URL = "http://noetl.noetl.svc.cluster.local:8082"
else:
    # Localhost configuration (port-forwarded from kind cluster)
    DB_CONFIG = {
        "host": "localhost",
        "port": "54321",  # Maps to postgres NodePort 30321
        "user": os.getenv("POSTGRES_USER", "demo"),
        "password": os.getenv("POSTGRES_PASSWORD", "demo"),
        "dbname": os.getenv("POSTGRES_DB", "demo_noetl")
    }
    NOETL_SERVER_URL = "http://localhost:8082"  # Maps to noetl NodePort 30082

TEST_PATH = "tests/pagination/loop_with_pagination/loop_with_pagination"
POLL_INTERVAL = 2
MAX_WAIT = 120

print("✓ Configuration loaded")
print(f"  Environment: {ENVIRONMENT}")
print(f"  Server: {NOETL_SERVER_URL}")
print(f"  Database: {DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['dbname']}")
print(f"  Test: {TEST_PATH}")


✓ Configuration loaded
  Environment: localhost
  Server: http://localhost:8082
  Database: localhost:54321/demo_noetl
  Test: tests/pagination/loop_with_pagination/loop_with_pagination


## 2. Initialize Test Table

In [2]:
# Create test schema and table - completely self-contained
import os
import psycopg

def get_postgres_connection():
    """Get psycopg3 connection"""
    conn_string = f"host={os.getenv('POSTGRES_HOST', 'postgres.postgres.svc.cluster.local')} " \
                  f"port={os.getenv('POSTGRES_PORT', '5432')} " \
                  f"dbname={os.getenv('POSTGRES_DB', 'demo_noetl')} " \
                  f"user={os.getenv('POSTGRES_USER', 'demo')} " \
                  f"password={os.getenv('POSTGRES_PASSWORD', 'demo')}"
    return psycopg.connect(conn_string)

create_table_sql = """
CREATE SCHEMA IF NOT EXISTS noetl_test;

DROP TABLE IF EXISTS noetl_test.pagination_loop_results;

CREATE TABLE noetl_test.pagination_loop_results (
    id SERIAL PRIMARY KEY,
    execution_id BIGINT,
    endpoint_name TEXT,
    endpoint_path TEXT,
    page_size INTEGER,
    result_count INTEGER,
    result_data JSONB,
    iteration_index INTEGER,
    iteration_count INTEGER,
    created_at TIMESTAMP DEFAULT NOW()
);

CREATE INDEX IF NOT EXISTS idx_pagination_loop_execution_id 
ON noetl_test.pagination_loop_results(execution_id);
"""

with get_postgres_connection() as conn:
    with conn.cursor() as cur:
        cur.execute(create_table_sql)
        conn.commit()

print("✓ Test table created")
print("  Schema: noetl_test")
print("  Table: pagination_loop_results")

✓ Test table created
  Schema: noetl_test
  Table: pagination_loop_results


## 3. Database Utilities

In [3]:
def get_postgres_connection():
    """Get psycopg3 connection"""
    conn_string = f"host={DB_CONFIG['host']} port={DB_CONFIG['port']} " \
                  f"dbname={DB_CONFIG['dbname']} user={DB_CONFIG['user']} " \
                  f"password={DB_CONFIG['password']}"
    return psycopg.connect(conn_string)

def query_to_polars(query: str) -> pl.DataFrame:
    """Execute query and return as Polars DataFrame"""
    with get_postgres_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            columns = [desc[0] for desc in cur.description]
            data = cur.fetchall()
    if not data:
        return pl.DataFrame(schema=columns)
    return pl.DataFrame({col: [row[i] for row in data] for i, col in enumerate(columns)})

print("✓ Database utilities loaded")

✓ Database utilities loaded


## 4. Execute Pagination Loop Test

In [4]:
def start_test() -> Dict:
    """Start pagination loop test"""
    url = f"{NOETL_SERVER_URL}/api/run/playbook"
    payload = {"path": TEST_PATH}
    
    print(f"Starting test: {TEST_PATH}")
    response = requests.post(url, json=payload, timeout=30)
    response.raise_for_status()
    
    result = response.json()
    execution_id = result['execution_id']
    
    print(f"✓ Test started")
    print(f"  Execution ID: {execution_id}")
    print(f"  Status: {result['status']}")
    
    return result

test_result = start_test()
EXECUTION_ID = test_result['execution_id']

Starting test: tests/pagination/loop_with_pagination/loop_with_pagination
✓ Test started
  Execution ID: 511672404708425883
  Status: running


## 5. Monitor Execution

In [5]:
def monitor_execution(execution_id: int):
    """Monitor test execution"""
    start_time = time.time()
    last_count = 0
    
    print(f"Monitoring execution {execution_id}...")
    print(f"{'Time':<6} {'Steps':<6} {'Status':<12} {'Events'}")
    print("-" * 50)
    
    while (time.time() - start_time) < MAX_WAIT:
        query = f"""
            SELECT event_type, COUNT(*) as count
            FROM noetl.event
            WHERE execution_id = {execution_id}
            GROUP BY event_type
        """
        df = query_to_polars(query)
        
        step_count = df.filter(pl.col('event_type') == 'step_completed')['count'].sum() or 0
        is_complete = df.filter(pl.col('event_type') == 'playbook_completed').height > 0
        is_failed = df.filter(pl.col('event_type') == 'playbook_failed').height > 0
        
        if step_count != last_count or is_complete or is_failed:
            elapsed = int(time.time() - start_time)
            status = "COMPLETED" if is_complete else ("FAILED" if is_failed else "RUNNING")
            total = df['count'].sum()
            print(f"{elapsed:<6} {step_count:<6} {status:<12} {total}")
            last_count = step_count
        
        if is_complete:
            print(f"\n✓ Test completed in {elapsed}s")
            return True
        elif is_failed:
            print(f"\n✗ Test failed after {elapsed}s")
            return False
        
        time.sleep(POLL_INTERVAL)
    
    print(f"\n⚠ Timeout after {MAX_WAIT}s")
    return False

success = monitor_execution(EXECUTION_ID)

Monitoring execution 511672404708425883...
Time   Steps  Status       Events
--------------------------------------------------

⚠ Timeout after 120s

⚠ Timeout after 120s


## 6. Validate Iterator Event Architecture

**Expected Behavior:**
1. Worker detects `loop` configuration in step
2. Routes to iterator executor 
3. Analyzes collection (filter, sort, limit)
4. Emits `iterator_started` event with metadata
5. Server should process event and enqueue iteration jobs (NOT YET IMPLEMENTED)

This section verifies the distributed loop architecture is working correctly.

In [6]:
# Validate iterator event architecture
print(f"\n📊 Iterator Event Validation for Execution {EXECUTION_ID}")
print("=" * 80)

# 1. Check basic event flow
events_query = f"""
    SELECT 
        event_type,
        node_name,
        status,
        created_at
    FROM noetl.event
    WHERE execution_id = {EXECUTION_ID}
    ORDER BY event_id
"""
events_df = query_to_polars(events_query)

print(f"\n✓ Event Flow ({events_df.height} events):")
for row in events_df.iter_rows(named=True):
    print(f"  {row['event_type']:25} {row['status']:12} {row['node_name'] or ''}")

# 2. Check iterator_started event exists
iterator_check = f"""
    SELECT 
        event_type,
        status,
        context->>'total_count' as total_count,
        context->>'mode' as mode,
        context->>'iterator_name' as iterator_name,
        jsonb_array_length(context->'collection') as collection_size,
        context->'nested_task'->>'tool' as nested_tool
    FROM noetl.event
    WHERE execution_id = {EXECUTION_ID}
      AND event_type = 'iterator_started'
"""
iterator_df = query_to_polars(iterator_check)

if iterator_df.height > 0:
    print(f"\n✓ iterator_started Event Found:")
    row = iterator_df.row(0, named=True)
    print(f"  Status:           {row['status']}")
    print(f"  Total Count:      {row['total_count']}")
    print(f"  Collection Size:  {row['collection_size']}")
    print(f"  Mode:             {row['mode']}")
    print(f"  Iterator Name:    {row['iterator_name']}")
    print(f"  Nested Tool:      {row['nested_tool']}")
else:
    print("\n❌ iterator_started event NOT FOUND!")
    print("   This means the event callback integration failed.")

# 3. Check iterator metadata
if iterator_df.height > 0:
    metadata_query = f"""
        SELECT 
            context->'collection' as collection,
            context->'nested_task' as nested_task
        FROM noetl.event
        WHERE execution_id = {EXECUTION_ID}
          AND event_type = 'iterator_started'
    """
    
    with get_postgres_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(metadata_query)
            result = cur.fetchone()
            if result:
                collection = result[0]
                nested_task = result[1]
                
                print(f"\n✓ Iterator Metadata:")
                print(f"  Collection: {json.dumps(collection, indent=2)[:200]}...")
                print(f"\n  Nested Task Tool: {nested_task.get('tool')}")
                print(f"  Has retry.on_success: {'retry' in nested_task and 'on_success' in nested_task.get('retry', {})}")
                
                if 'retry' in nested_task and 'on_success' in nested_task['retry']:
                    retry_config = nested_task['retry']['on_success']
                    print(f"  Pagination Config:")
                    print(f"    - while: {retry_config.get('while', 'N/A')[:60]}")
                    print(f"    - max_attempts: {retry_config.get('max_attempts', 'N/A')}")
                    print(f"    - collect.strategy: {retry_config.get('collect', {}).get('strategy', 'N/A')}")

# 4. Check for expected next events (iteration jobs)
iteration_check = f"""
    SELECT COUNT(*) as count
    FROM noetl.event
    WHERE execution_id = {EXECUTION_ID}
      AND event_type = 'iteration_completed'
"""
iteration_df = query_to_polars(iteration_check)

if iteration_df['count'][0] > 0:
    print(f"\n✓ Found {iteration_df['count'][0]} iteration_completed events")
    print("  Server successfully processed iterator_started and enqueued iteration jobs!")
else:
    print(f"\n⚠ No iteration_completed events found")
    print("  This is EXPECTED - Server orchestrator doesn't yet process iterator_started")
    print("  Next Implementation Step: Add _process_iterator_started() handler in orchestrator.py")

# 5. Summary
print(f"\n{'='*80}")
print("VALIDATION SUMMARY:")
print("=" * 80)

if iterator_df.height > 0:
    print("✅ Worker Event Callback: WORKING")
    print("✅ Iterator Executor: WORKING") 
    print("✅ iterator_started Event: EMITTED")
    print("✅ Event Schema: VALID")
    print("⏳ Server Orchestrator: NOT YET IMPLEMENTED")
    print("\nNext: Implement server-side iterator_started event processing to enqueue iteration jobs")
else:
    print("❌ Iterator event architecture validation FAILED")
    print("   Check worker logs for event emission errors")


📊 Iterator Event Validation for Execution 511672404708425883

✓ Event Flow (7 events):
  playbook_started          STARTED      tests/pagination/loop_with_pagination/loop_with_pagination
  workflow_initialized      COMPLETED    workflow
  step_started              RUNNING      fetch_all_endpoints
  action_started            RUNNING      fetch_all_endpoints
  iterator_started          RUNNING      iterator
  action_completed          COMPLETED    fetch_all_endpoints
  step_result               COMPLETED    fetch_all_endpoints

✓ iterator_started Event Found:
  Status:           RUNNING
  Total Count:      2
  Collection Size:  2
  Mode:             sequential
  Iterator Name:    endpoint
  Nested Tool:      http

✓ Iterator Metadata:
  Collection: [
  {
    "name": "assessments",
    "path": "/api/v1/assessments",
    "page_size": 10
  },
  {
    "name": "users",
    "path": "/api/v1/users",
    "page_size": 15
  }
]...

  Nested Task Tool: http
  Has retry.on_success: True
  Paginatio

## 7. Architecture Status & Next Steps

**✅ Completed Implementation:**
- Worker-side event callback integration
- Iterator executor analysis and event emission  
- Event schema extensions (iterator_started, iterator_completed, etc.)
- Environment-aware configuration (localhost/kubernetes)

**⏳ Pending Implementation:**
- Server orchestrator `_process_iterator_started()` handler
- Iteration job enqueueing logic
- Iteration execution with pagination (retry.on_success)
- `iterator_completed` event emission after all iterations

**Note:** The test will timeout until server-side processing is implemented.

## Summary: Distributed Loop + Pagination Architecture

**What This Test Validates:**

This notebook demonstrates the **event-driven distributed loop architecture** where:
- Workers analyze collections and emit events
- Server processes events to orchestrate distributed execution
- Each iteration runs independently with pagination support

**Current Implementation Status:**

```
✅ PHASE 1: Worker-Side Architecture (COMPLETE)
   ├─ Loop detection in execute_task()
   ├─ Iterator executor analysis (filter, sort, limit)
   ├─ Event callback integration
   └─ iterator_started event emission

⏳ PHASE 2: Server-Side Orchestration (PENDING)
   ├─ Process iterator_started event
   ├─ Enqueue N iteration jobs
   ├─ Track iteration completion
   └─ Emit iterator_completed event

🔮 PHASE 3: Pagination via Retry (DESIGN READY)
   ├─ HTTP action executes with retry.on_success
   ├─ Server-side pagination state tracking
   ├─ Page continuation logic
   └─ Result aggregation
```

**How to Use:**

1. **Run locally:** Execute cells 1-6 (default: localhost mode)
2. **Run in-cluster:** Set `NOETL_ENV=kubernetes` before cell 2
3. **Check validation:** Cell 6 shows full architecture validation

**Next Steps:**

Implement `_process_iterator_started()` in `orchestrator.py` to:
- Parse collection from iterator_started.context
- Create N PreparedJob instances
- Enqueue to noetl.queue table
- Workers will pick up and execute with pagination

## Debug: Check Server Logs

In [7]:
# Check server logs for orchestrator errors
import subprocess

execution_id = 511672404708425883

# Get server pod name
result = subprocess.run(
    ["kubectl", "get", "pods", "-n", "noetl", "-l", "app=noetl-server", "-o", "jsonpath={.items[0].metadata.name}"],
    capture_output=True,
    text=True
)
server_pod = result.stdout.strip()
print(f"Server pod: {server_pod}\n")

# Check logs around the execution time
result = subprocess.run(
    ["kubectl", "logs", "-n", "noetl", server_pod, "--tail=200"],
    capture_output=True,
    text=True
)

# Filter for ORCHESTRATOR messages and errors
lines = result.stdout.split('\n')
relevant_lines = [
    line for line in lines 
    if 'ORCHESTRATOR' in line or 'ERROR' in line or 'KeyError' in line or str(execution_id) in line
]

print(f"Found {len(relevant_lines)} relevant log lines:\n")
for line in relevant_lines[-30:]:  # Last 30 relevant lines
    print(line)

Server pod: noetl-server-bc75cf667-m7bph

Found 0 relevant log lines:



## Check Queue Table for Iteration Jobs

In [8]:
# Check queue table for iteration jobs
execution_id = 511672404708425883

with psycopg.connect(**{k: v for k, v in DB_CONFIG.items()}) as conn:
    with conn.cursor() as cur:
        # Check for iteration jobs in queue
        cur.execute("""
            SELECT 
                execution_id,
                parent_execution_id,
                node_name,
                status,
                created_at
            FROM noetl.queue
            WHERE parent_execution_id = %s OR execution_id = %s
            ORDER BY created_at
        """, (execution_id, execution_id))
        
        jobs = cur.fetchall()
        
        print(f"Queue entries for execution {execution_id}:")
        print(f"Total jobs: {len(jobs)}\n")
        
        if jobs:
            print(f"{'Execution ID':<25} {'Parent':<25} {'Node':<30} {'Status':<15} {'Created'}")
            print("-" * 130)
            for job in jobs:
                exec_id, parent_id, node, status, created = job
                print(f"{exec_id:<25} {parent_id or 'None':<25} {node:<30} {status:<15} {created}")
        else:
            print("⚠ No jobs found in queue!")
            print("\nThis means the orchestrator did NOT enqueue iteration jobs.")
            print("Let's check if the iterator_started event was processed at all...")

Queue entries for execution 511672404708425883:
Total jobs: 1

Execution ID              Parent                    Node                           Status          Created
----------------------------------------------------------------------------------------------------------------------------------
511672404708425883        None                      fetch_all_endpoints            done            2025-12-06 23:21:43.880866+00:00


## Check If Orchestrator is Processing iterator_started

In [9]:
# Get ALL server logs and search for any orchestrator activity
import subprocess

result = subprocess.run(
    ["kubectl", "get", "pods", "-n", "noetl", "-l", "app=noetl-server", "-o", "jsonpath={.items[0].metadata.name}"],
    capture_output=True,
    text=True
)
server_pod = result.stdout.strip()

result = subprocess.run(
    ["kubectl", "logs", "-n", "noetl", server_pod, "--tail=500"],
    capture_output=True,
    text=True
)

lines = result.stdout.split('\n')

# Look for any of these patterns
patterns = ['ORCHESTRATOR', 'iterator_started', 'Enqueueing', '_process_iterator', 'evaluate_execution']

print(f"Searching {len(lines)} log lines for orchestrator activity...\n")

matches = []
for i, line in enumerate(lines):
    for pattern in patterns:
        if pattern in line:
            matches.append((i, line))
            break

print(f"Found {len(matches)} relevant lines:\n")
for idx, line in matches[-20:]:  # Last 20 matches
    print(f"[{idx:4d}] {line}")

Searching 501 log lines for orchestrator activity...

Found 0 relevant lines:



## Check Postgres NOTIFY/LISTEN Configuration

In [10]:
# Check if NOTIFY triggers are configured for iterator_started events
with psycopg.connect(**{k: v for k, v in DB_CONFIG.items()}) as conn:
    with conn.cursor() as cur:
        # Check trigger function
        cur.execute("""
            SELECT 
                trigger_name,
                event_manipulation,
                action_statement
            FROM information_schema.triggers
            WHERE trigger_schema = 'noetl'
            AND event_object_table = 'event'
        """)
        
        triggers = cur.fetchall()
        print(f"Triggers on noetl.event table: {len(triggers)}\n")
        
        for trig in triggers:
            name, event, action = trig
            print(f"Trigger: {name}")
            print(f"  Event: {event}")
            print(f"  Action: {action[:200]}...")
            print()
        
        # Check the trigger function definition
        cur.execute("""
            SELECT pg_get_functiondef(oid)
            FROM pg_proc
            WHERE proname = 'notify_event_channel'
            AND pronamespace = (SELECT oid FROM pg_namespace WHERE nspname = 'noetl')
        """)
        
        func_def = cur.fetchone()
        if func_def:
            print("\nTrigger Function Definition:")
            print(func_def[0])

Triggers on noetl.event table: 0



## Check If evaluate_execution Was Called For iterator_started

In [11]:
# Test if orchestrator can be triggered manually for the iterator_started event
execution_id = 511672404708425883

# Get the iterator_started event
with psycopg.connect(**{k: v for k, v in DB_CONFIG.items()}) as conn:
    with conn.cursor() as cur:
        cur.execute("""
            SELECT event_id, event_type
            FROM noetl.event
            WHERE execution_id = %s
            AND event_type = 'iterator_started'
        """, (execution_id,))
        
        event = cur.fetchone()
        
        if event:
            event_id, event_type = event
            print(f"Found iterator_started event: {event_id}")
            print(f"\nManually triggering orchestrator...")
            
            # Call the orchestration endpoint directly
            response = requests.post(
                f"{NOETL_SERVER_URL}/api/orchestrate",
                json={
                    "execution_id": str(execution_id),
                    "trigger_event_type": event_type,
                    "trigger_event_id": str(event_id)
                }
            )
            
            print(f"Response status: {response.status_code}")
            if response.status_code == 200:
                print(f"✓ Orchestrator triggered successfully")
                print(f"Response: {response.json()}")
            else:
                print(f"✗ Error: {response.text}")
        else:
            print("No iterator_started event found!")

Found iterator_started event: 511672435494617249

Manually triggering orchestrator...
Response status: 405
✗ Error: {"detail":"Method Not Allowed"}


## Solution: Check Orchestrator Code Path

In [12]:
# The issue: evaluate_execution has special handling for iterator_started
# but it appears it's not being called. Let me check the flow:
# 1. Worker emits iterator_started via POST /api/events
# 2. emit_event_legacy calls emit_event  
# 3. emit_event calls evaluate_execution with trigger_event_type="iterator_started"
# 4. evaluate_execution should check for iterator_started and call _process_iterator_started

# BUT: There are NO logs showing "ORCHESTRATOR: Evaluating" which means evaluate_execution is NOT being called
# Let's verify the event was actually emitted by checking if it's in the event table

execution_id = 511672404708425883

with psycopg.connect(**{k: v for k, v in DB_CONFIG.items()}) as conn:
    with conn.cursor() as cur:
        # Get all events for this execution
        cur.execute("""
            SELECT 
                event_type,
                status,
                created_at,
                event_id
            FROM noetl.event
            WHERE execution_id = %s
            ORDER BY created_at
        """, (execution_id,))
        
        events = cur.fetchall()
        
        print(f"Events for execution {execution_id}:")
        print(f"{'Event Type':<25} {'Status':<15} {'Created':<30} {'Event ID'}")
        print("-" * 100)
        for event_type, status, created, event_id in events:
            marker = " ← SHOULD TRIGGER ORCHESTRATOR" if event_type == "iterator_started" else ""
            print(f"{event_type:<25} {status:<15} {str(created):<30} {event_id}{marker}")
        
        print("\n🔍 Root Cause Analysis:")
        print("========================")
        print("The iterator_started event exists, but there are NO orchestrator logs.")
        print("This means one of two things:")
        print("1. The emit_event endpoint is NOT calling evaluate_execution")
        print("2. The evaluate_execution function is silently failing/returning early")
        print("\nLet's check the server code to see which one...")

Events for execution 511672404708425883:
Event Type                Status          Created                        Event ID
----------------------------------------------------------------------------------------------------
playbook_started          STARTED         2025-12-06 23:21:43.861318     511672404817477788
workflow_initialized      COMPLETED       2025-12-06 23:21:43.874872     511672404934918301
step_started              RUNNING         2025-12-06 23:21:43.884671     511672405010415775
action_started            RUNNING         2025-12-06 23:21:47.461532     511672435016466592
iterator_started          RUNNING         2025-12-06 23:21:47.519055     511672435494617249 ← SHOULD TRIGGER ORCHESTRATOR
action_completed          COMPLETED       2025-12-06 23:21:47.530429     511672435595280546
step_result               COMPLETED       2025-12-06 23:21:47.542893     511672435704332451

🔍 Root Cause Analysis:
The iterator_started event exists, but there are NO orchestrator logs.
This me

## Manual Debug: Emit Test Event and Watch Logs

In [13]:
# Emit a test event and check if evaluate_execution is called
import time
import subprocess

test_exec_id = 999999999999999999  # Test execution ID

print("Emitting test event...")
response = requests.post(
    f"{NOETL_SERVER_URL}/api/event/emit",
    json={
        "execution_id": str(test_exec_id),
        "event_type": "iterator_started",
        "status": "RUNNING",
        "context": {"test": "manual emit"}
    }
)

print(f"Response: {response.status_code}")
if response.status_code == 200:
    print(f"Event emitted: {response.json()}")
else:
    print(f"Error: {response.text}")

# Wait a moment for log flush
time.sleep(2)

# Check logs
result = subprocess.run(
    ["kubectl", "get", "pods", "-n", "noetl", "-l", "app=noetl-server", "-o", "jsonpath={.items[0].metadata.name}"],
    capture_output=True,
    text=True
)
server_pod = result.stdout.strip()

result = subprocess.run(
    ["kubectl", "logs", "-n", "noetl", server_pod, "--tail=50"],
    capture_output=True,
    text=True
)

lines = result.stdout.split('\n')
print(f"\n📋 Last 50 log lines (filtered for test execution):")
for line in lines:
    if str(test_exec_id) in line or 'ORCHESTRATOR' in line or 'EMIT EVENT' in line:
        print(line)

Emitting test event...
Response: 500
Error: {"detail":"null value in column \"catalog_id\" of relation \"event\" violates not-null constraint\nDETAIL:  Failing row contains (999999999999999999, null, 511675646569873572, null, null, 2025-12-06 23:28:10.311218, iterator_started, null, null, null, RUNNING, null, {\"test\": \"manual emit\"}, null, null, null, null, null, null, null, null, null, null, null)."}

📋 Last 50 log lines (filtered for test execution):
     Message: EMIT EVENT: execution_id=999999999999999999, type=iterator_started
     Message: Failed to resolve catalog_id for execution 999999999999999999: No catalog_id found for execution 999999999999999999
     Message: Error emitting event for execution_id=999999999999999999, event_type=iterator_started: null value in column "catalog_id" of relation "event" violates not-null constraint
             DETAIL:  Failing row contains (999999999999999999, null, 511675646569873572, null, null, 2025-12-06 23:28:10.311218, iterator_start

## Check Server Logs From Real Execution Time

In [14]:
# The iterator_started event was emitted at 2025-12-06 23:21:47.519055
# Let's check ALL server logs from around that time

execution_id = 511672404708425883

result = subprocess.run(
    ["kubectl", "get", "pods", "-n", "noetl", "-l", "app=noetl-server", "-o", "jsonpath={.items[0].metadata.name}"],
    capture_output=True,
    text=True
)
server_pod = result.stdout.strip()

# Get logs with timestamps - look for logs around 23:21:47
result = subprocess.run(
    ["kubectl", "logs", "-n", "noetl", server_pod, "--timestamps", "--since=10m"],
    capture_output=True,
    text=True
)

lines = result.stdout.split('\n')

# Filter for logs related to our execution or iterator/orchestrator
print(f"Searching logs for execution {execution_id}...\n")

relevant = []
for line in lines:
    if str(execution_id) in line:
        relevant.append(line)
    elif any(word in line for word in ['iterator_started', 'ORCHESTRATOR', 'EMIT EVENT']):
        # Check if timestamp is around 23:21
        if '23:21:' in line or '23:22:' in line:
            relevant.append(line)

print(f"Found {len(relevant)} relevant lines:\n")
for line in relevant[-40:]:  # Last 40 relevant lines
    print(line)

Searching logs for execution 511672404708425883...

Found 66 relevant lines:

2025-12-06T23:21:47.469639261Z      Message: ORCHESTRATOR: Dispatching initial step for execution 511672404708425883
2025-12-06T23:21:47.469643511Z      Message: Dispatching first step for execution 511672404708425883
2025-12-06T23:21:47.469647803Z      Message: ORCHESTRATOR: Evaluation complete for execution 511672404708425883
2025-12-06T23:21:47.469879428Z                "execution_id": "511672404708425883",
2025-12-06T23:21:47.469990678Z                    "execution_id": 511672404708425883
2025-12-06T23:21:47.470191970Z                    "execution_id": 511672404708425883
2025-12-06T23:21:47.470231470Z                "execution_id": "511672404708425883",
2025-12-06T23:21:47.517976053Z      Message: EMIT EVENT: execution_id=511672404708425883, type=iterator_started
2025-12-06T23:21:47.519119261Z      Message: Resolved missing catalog_id=511594959275819078 from execution_id=511672404708425883
2025-12-06T23

## Get Full Error Traceback

In [15]:
# Get full logs with tracebacks around the iterator_started error

result = subprocess.run(
    ["kubectl", "get", "pods", "-n", "noetl", "-l", "app=noetl-server", "-o", "jsonpath={.items[0].metadata.name}"],
    capture_output=True,
    text=True
)
server_pod = result.stdout.strip()

# Get logs with JSON formatting to see full messages
result = subprocess.run(
    ["kubectl", "logs", "-n", "noetl", server_pod, "--timestamps", "--since=10m"],
    capture_output=True,
    text=True
)

lines = result.stdout.split('\n')

# Find the error around iterator_started
print("Looking for error traceback around iterator_started processing...\n")

in_error_block = False
error_lines = []
for i, line in enumerate(lines):
    if '511672435494617249' in line and 'Detected iterator_started' in line:
        # Start capturing from here
        in_error_block = True
        error_lines.append(line)
    elif in_error_block:
        error_lines.append(line)
        # Stop after we see the error and a few more lines
        if len(error_lines) > 50:
            break

print("Error context:")
print("=" * 100)
for line in error_lines[:50]:
    print(line)

Looking for error traceback around iterator_started processing...

Error context:


## Run New Test Execution With Fixed Orchestrator

In [16]:
# Run a fresh test execution now that the fixed orchestrator is deployed
print("🚀 Launching test execution with fixed orchestrator...\n")

response = requests.post(
    f"{NOETL_SERVER_URL}/api/run/playbook",
    json={
        "path": TEST_PATH,
        "workload": {}
    }
)

if response.status_code == 200:
    result = response.json()
    new_exec_id = result['execution_id']
    print(f"✓ Execution started: {new_exec_id}")
    print(f"  Status: {result.get('status')}\n")
    
    # Wait a moment for processing
    print("Waiting for orchestrator to process iterator_started...")
    time.sleep(5)
    
    # Check queue for iteration jobs
    with psycopg.connect(**{k: v for k, v in DB_CONFIG.items()}) as conn:
        with conn.cursor() as cur:
            cur.execute("""
                SELECT 
                    execution_id,
                    parent_execution_id,
                    node_name,
                    status
                FROM noetl.queue
                WHERE parent_execution_id = %s OR execution_id = %s
                ORDER BY created_at
            """, (new_exec_id, new_exec_id))
            
            jobs = cur.fetchall()
            
            print(f"\n📋 Queue entries (expecting 1 parent + 2 iterations = 3 total):")
            print(f"   Found: {len(jobs)} jobs\n")
            
            if len(jobs) > 0:
                print(f"{'Execution ID':<25} {'Parent':<25} {'Node':<40} {'Status'}")
                print("-" * 120)
                for exec_id, parent_id, node, status in jobs:
                    print(f"{exec_id:<25} {parent_id or '-':<25} {node:<40} {status}")
                    
                if len(jobs) >= 3:
                    print("\n🎉 SUCCESS! Iteration jobs were enqueued!")
                    print("   Phase 2 orchestration is working!")
                else:
                    print(f"\n⚠ Expected 3 jobs (1 parent + 2 iterations), got {len(jobs)}")
            else:
                print("⚠ No jobs found - checking server logs for errors...")
else:
    print(f"✗ Failed to start execution: {response.status_code}")
    print(f"  Error: {response.text}")

🚀 Launching test execution with fixed orchestrator...

✓ Execution started: 511676708500537509
  Status: running

Waiting for orchestrator to process iterator_started...

📋 Queue entries (expecting 1 parent + 2 iterations = 3 total):
   Found: 1 jobs

Execution ID              Parent                    Node                                     Status
------------------------------------------------------------------------------------------------------------------------
511676708500537509        -                         fetch_all_endpoints                      done

⚠ Expected 3 jobs (1 parent + 2 iterations), got 1


## Final Test After Pod Restart

In [17]:
# Final test after forcing pod restart
print("🚀 Final test with freshly restarted pod...\n")

response = requests.post(
    f"{NOETL_SERVER_URL}/api/run/playbook",
    json={
        "path": TEST_PATH
    }
)

if response.status_code == 200:
    result = response.json()
    final_exec_id = result['execution_id']
    print(f"✓ Execution started: {final_exec_id}\n")
    
    # Wait for processing
    print("Waiting 5 seconds...")
    time.sleep(5)
    
    # Check queue
    with psycopg.connect(**{k: v for k, v in DB_CONFIG.items()}) as conn:
        with conn.cursor() as cur:
            cur.execute("""
                SELECT 
                    execution_id,
                    parent_execution_id,
                    node_name,
                    status
                FROM noetl.queue
                WHERE parent_execution_id = %s OR execution_id = %s
                ORDER BY created_at
            """, (final_exec_id, final_exec_id))
            
            jobs = cur.fetchall()
            
            print(f"\n📋 Queue Status:")
            print(f"   Total jobs: {len(jobs)}\n")
            
            if len(jobs) > 0:
                print(f"{'Execution ID':<25} {'Parent':<25} {'Node':<45} {'Status'}")
                print("-" * 130)
                for exec_id, parent_id, node, status in jobs:
                    marker = " ✓" if parent_id else ""
                    print(f"{exec_id:<25} {parent_id or '-':<25} {node:<45} {status}{marker}")
                    
                iteration_count = sum(1 for _, parent_id, _, _ in jobs if parent_id)
                print(f"\n   Parent job: 1")
                print(f"   Iteration jobs: {iteration_count}")
                
                if iteration_count == 2:
                    print("\n🎉🎉🎉 SUCCESS! Phase 2 is working! 🎉🎉🎉")
                    print("   - Server detected iterator_started")
                    print("   - Enqueued 2 iteration jobs")
                    print("   - Workers can now execute iterations with pagination")
else:
    print(f"✗ Error: {response.status_code} - {response.text}")

🚀 Final test with freshly restarted pod...

✓ Execution started: 511677392524411054

Waiting 5 seconds...

📋 Queue Status:
   Total jobs: 1

Execution ID              Parent                    Node                                          Status
----------------------------------------------------------------------------------------------------------------------------------
511677392524411054        -                         fetch_all_endpoints                           done

   Parent job: 1
   Iteration jobs: 0


## Complete Deployment and Test

In [18]:
# Complete image load and deployment
import subprocess
import time

print("Loading image into kind...")
result = subprocess.run(
    ["kind", "load", "docker-image", "local/noetl:2025-12-06-15-33", "--name", "noetl"],
    capture_output=True,
    text=True,
    timeout=120
)

if result.returncode == 0:
    print("✓ Image loaded\n")
    
    print("Restarting deployments...")
    subprocess.run(["kubectl", "rollout", "restart", "deployment", "-n", "noetl", "noetl-server", "noetl-worker"])
    
    print("Waiting for rollout...")
    time.sleep(10)
    
    result = subprocess.run(
        ["kubectl", "get", "pods", "-n", "noetl"],
        capture_output=True,
        text=True
    )
    print(result.stdout)
    
    print("\n✓ Deployment complete! Ready to test.")
else:
    print(f"✗ Image load failed: {result.stderr}")

Loading image into kind...
✓ Image loaded

Restarting deployments...
deployment.apps/noetl-server restarted
deployment.apps/noetl-worker restarted
Waiting for rollout...
NAME                            READY   STATUS        RESTARTS   AGE
noetl-server-5bddfb74ff-cpds5   1/1     Running       0          10s
noetl-worker-6d8cd96b68-v4gwv   1/1     Running       0          10s
noetl-worker-8697858789-66kz8   1/1     Terminating   0          11s


✓ Deployment complete! Ready to test.


In [19]:
# Run final test with all fixes deployed
print("Launching test execution...")
response = requests.post(
    f"{NOETL_SERVER_URL}/api/run/playbook",
    json={
        "path": TEST_PATH
    }
)

if response.status_code == 200:
    result = response.json()
    exec_id = result['execution_id']
    print(f"✓ Test started: {exec_id}\n")
    
    # Wait a moment for processing
    time.sleep(3)
    
    # Check queue for iteration jobs
    with psycopg.connect(**{k: v for k, v in DB_CONFIG.items()}) as conn:
        with conn.cursor() as cur:
            cur.execute("""
                SELECT execution_id, parent_execution_id, node_name, status
                FROM noetl.queue
                WHERE execution_id = %s OR parent_execution_id = %s
                ORDER BY created_at
            """, (exec_id, exec_id))
            
            jobs = cur.fetchall()
            
            print(f"Queue entries: {len(jobs)}")
            print(f"{'Execution ID':<25} {'Parent':<25} {'Node':<40} {'Status'}")
            print("-" * 120)
            for job in jobs:
                job_id, parent, node, status = job
                marker = " ← ITERATION JOB!" if parent == exec_id else ""
                print(f"{job_id:<25} {parent or 'None':<25} {node:<40} {status}{marker}")
            
            if len(jobs) > 1:
                print(f"\n🎉 SUCCESS! Found {len(jobs)-1} iteration jobs enqueued!")
                print("Phase 2 is WORKING!")
            else:
                print("\n⚠ No iteration jobs found yet. Checking logs...")
else:
    print(f"✗ Failed to start: {response.status_code} - {response.text}")

Launching test execution...
✓ Test started: 511679072586432695

Queue entries: 1
Execution ID              Parent                    Node                                     Status
------------------------------------------------------------------------------------------------------------------------
511679072586432695        None                      fetch_all_endpoints                      leased

⚠ No iteration jobs found yet. Checking logs...


In [20]:
# Check logs for THIS execution
exec_id = 511679072586432695

result = subprocess.run(
    ["kubectl", "get", "pods", "-n", "noetl", "-l", "app=noetl-server", "-o", "jsonpath={.items[0].metadata.name}"],
    capture_output=True,
    text=True
)
server_pod = result.stdout.strip()

result = subprocess.run(
    ["kubectl", "logs", "-n", "noetl", server_pod, "--tail=100"],
    capture_output=True,
    text=True
)

lines = result.stdout.split('\n')

print(f"Searching for execution {exec_id} or iterator_started...\n")
for line in lines:
    if str(exec_id) in line or ('iterator_started' in line and '23:' in line):
        print(line)

Searching for execution 511679072586432695 or iterator_started...



In [21]:
# Wait for worker to process and emit iterator_started
exec_id = 511679072586432695

print("Waiting for worker to complete...")
time.sleep(5)

# Check events
with psycopg.connect(**{k: v for k, v in DB_CONFIG.items()}) as conn:
    with conn.cursor() as cur:
        cur.execute("""
            SELECT event_type, status, created_at
            FROM noetl.event
            WHERE execution_id = %s
            ORDER BY created_at
        """, (exec_id,))
        
        events = cur.fetchall()
        
        print(f"\nEvents for execution {exec_id}:")
        for event_type, status, created in events:
            print(f"  {event_type:<25} {status:<15} {created}")
        
        # Check for iterator_started
        has_iterator_started = any(e[0] == 'iterator_started' for e in events)
        
        if has_iterator_started:
            print("\n✓ iterator_started event emitted!")
            print("Waiting 2 more seconds for orchestrator...")
            time.sleep(2)
            
            # Check queue again
            cur.execute("""
                SELECT COUNT(*) 
                FROM noetl.queue
                WHERE parent_execution_id = %s
            """, (exec_id,))
            
            count = cur.fetchone()[0]
            print(f"Iteration jobs in queue: {count}")
        else:
            print("\n⏳ Still waiting for iterator_started event...")

Waiting for worker to complete...

Events for execution 511679072586432695:
  playbook_started          STARTED         2025-12-06 23:34:58.731446
  workflow_initialized      COMPLETED       2025-12-06 23:34:58.744419
  step_started              RUNNING         2025-12-06 23:34:58.754355

⏳ Still waiting for iterator_started event...


In [23]:
# Wait longer for worker
exec_id = 511679072586432695

found = False
for i in range(10):
    time.sleep(2)
    
    with psycopg.connect(**{k: v for k, v in DB_CONFIG.items()}) as conn:
        with conn.cursor() as cur:
            cur.execute("""
                SELECT event_type
                FROM noetl.event
                WHERE execution_id = %s
                AND event_type = 'iterator_started'
            """, (exec_id,))
            
            if cur.fetchone():
                found = True
                print(f"✓ iterator_started found after {(i+1)*2} seconds!")
                
                # Wait for orchestrator
                time.sleep(2)
                
                # Check queue
                cur.execute("""
                    SELECT execution_id, node_name
                    FROM noetl.queue
                    WHERE parent_execution_id = %s
                """, (exec_id,))
                
                iter_jobs = cur.fetchall()
                
                if iter_jobs:
                    print(f"\n🎉 SUCCESS! Found {len(iter_jobs)} iteration jobs:")
                    for job_id, node in iter_jobs:
                        print(f"  - {job_id}: {node}")
                    print("\nPhase 2 is WORKING! ✅")
                else:
                    print("\n⚠ No iteration jobs found. Checking logs for errors...")
                    
                    result = subprocess.run(
                        ["kubectl", "logs", "-n", "noetl", "-l", "app=noetl-server", "--tail=50"],
                        capture_output=True,
                        text=True
                    )
                    
                    for line in result.stdout.split('\n'):
                        if 'ERROR' in line or 'Traceback' in line or str(exec_id) in line:
                            print(line)
                
                break
    
    if not found:
        print(f"Waiting... ({(i+1)*2}s)")

if not found:
    print("\n⏸ Timeout waiting for iterator_started")

Waiting... (2s)
✓ iterator_started found after 4 seconds!

⚠ No iteration jobs found. Checking logs for errors...
               "execution_id": "511679072586432695",
     Message: QUEUE_COMPLETION_DEBUG: Job 511679072829702330 completed for execution 511679072586432695
     Message: ORCHESTRATOR: Evaluating execution_id=511679072586432695, trigger=None, event_id=None
     Message: ORCHESTRATOR: Execution 511679072586432695 state=in_progress
     Message: ORCHESTRATOR: Evaluation complete for execution 511679072586432695


In [24]:
# Get FULL logs for this execution to see what happened
exec_id = 511679072586432695

result = subprocess.run(
    ["kubectl", "logs", "-n", "noetl", "-l", "app=noetl-server", "--tail=200"],
    capture_output=True,
    text=True
)

lines = result.stdout.split('\n')

print(f"All logs mentioning execution {exec_id}:\n")
for line in lines:
    if str(exec_id) in line:
        print(line)

All logs mentioning execution 511679072586432695:

     Message: EMIT EVENT: execution_id=511679072586432695, type=step_result
     Message: Event emitted: event_id=511679633775919295, execution_id=511679072586432695, type=step_result, status=COMPLETED
     Message: ORCHESTRATOR: Evaluating execution_id=511679072586432695, trigger=step_result, event_id=511679633775919295
     Message: ORCHESTRATOR: Execution 511679072586432695 state=in_progress
     Message: ORCHESTRATOR: Error evaluating execution 511679072586432695
               "execution_id": "511679072586432695",
               "execution_id": "511679072586432695",
     Message: QUEUE_COMPLETION_DEBUG: Job 511679072829702330 completed for execution 511679072586432695
     Message: ORCHESTRATOR: Evaluating execution_id=511679072586432695, trigger=None, event_id=None
     Message: ORCHESTRATOR: Execution 511679072586432695 state=in_progress
     Message: ORCHESTRATOR: Evaluation complete for execution 511679072586432695


## Deploy Fixed Image and Test Phase 2

In [26]:
# Update deployment with latest image (2025-12-06-15-35) and test
import subprocess
import time

# Get latest image tag
with open('/Users/akuksin/projects/noetl/noetl/.noetl_last_build_tag.txt') as f:
    image_tag = f.read().strip()

print(f"Updating deployments to use: local/noetl:{image_tag}\n")

# Update deployments
for deployment, container in [('noetl-server', 'noetl-server'), ('noetl-worker', 'worker')]:
    result = subprocess.run(
        ['kubectl', 'set', 'image', f'deployment/{deployment}',
         f'{container}=local/noetl:{image_tag}', '-n', 'noetl'],
        capture_output=True,
        text=True
    )
    print(f"{deployment}: {result.stdout.strip() or result.stderr.strip()}")

# Wait for rollout
print("\nWaiting for rollout...")
time.sleep(15)

result = subprocess.run(
    ['kubectl', 'get', 'pods', '-n', 'noetl'],
    capture_output=True,
    text=True
)
print(result.stdout)

print("\n✅ Deployment complete! Running test...\n" + "="*60)

# Run test
response = requests.post(f"{NOETL_SERVER_URL}/api/run/playbook", json={"path": TEST_PATH})

if response.status_code == 200:
    test_exec_id = response.json()['execution_id']
    print(f"✓ Test execution: {test_exec_id}\n")
    
    # Wait and poll for iterator_started + iteration jobs
    for i in range(15):
        time.sleep(2)
        
        with psycopg.connect(**{k: v for k, v in DB_CONFIG.items()}) as conn:
            with conn.cursor() as cur:
                # Check for iterator_started
                cur.execute("SELECT 1 FROM noetl.event WHERE execution_id = %s AND event_type = 'iterator_started'", (test_exec_id,))
                if cur.fetchone():
                    print(f"✓ iterator_started emitted ({(i+1)*2}s)")
                    
                    time.sleep(1)
                    
                    # Check for iteration jobs
                    cur.execute("SELECT execution_id, node_name FROM noetl.queue WHERE parent_execution_id = %s", (test_exec_id,))
                    jobs = cur.fetchall()
                    
                    if jobs:
                        print(f"\n🎉🎉🎉 PHASE 2 SUCCESS! 🎉🎉🎉\n")
                        print(f"Found {len(jobs)} iteration jobs enqueued:")
                        for job_id, node in jobs:
                            print(f"  • {node}")
                        print("\n✅ Server orchestration working!")
                        print("✅ Iteration jobs properly enqueued!")
                        break
                    else:
                        print("⚠ No iteration jobs yet, checking logs...")
                        result = subprocess.run(
                            ['kubectl', 'logs', '-n', 'noetl', '-l', 'app=noetl-server', '--tail=30'],
                            capture_output=True, text=True
                        )
                        for line in result.stdout.split('\n'):
                            if 'ERROR' in line or str(test_exec_id) in line:
                                print(f"  {line}")
                        break
                        
        print(f"Waiting... ({(i+1)*2}s)")
else:
    print(f"✗ Failed: {response.status_code} - {response.text}")

Updating deployments to use: local/noetl:2025-12-06-15-38

noetl-server: deployment.apps/noetl-server image updated
noetl-worker: deployment.apps/noetl-worker image updated

Waiting for rollout...
NAME                            READY   STATUS    RESTARTS   AGE
noetl-server-7445bc4584-plkh8   1/1     Running   0          15s
noetl-worker-687565c5c7-h9pqj   1/1     Running   0          15s


✅ Deployment complete! Running test...
✓ Test execution: 511682789486362816

Waiting... (2s)
✓ iterator_started emitted (4s)
⚠ No iteration jobs yet, checking logs...
                 "execution_id": "511682789486362816",
       Message: QUEUE_COMPLETION_DEBUG: Job 511682789855461571 completed for execution 511682789486362816
       Message: ORCHESTRATOR: Evaluating execution_id=511682789486362816, trigger=None, event_id=None
       Message: ORCHESTRATOR: Execution 511682789486362816 state=in_progress
       Message: ORCHESTRATOR: Evaluation complete for execution 511682789486362816


In [36]:
# FINAL TEST: Phase 2 validation with fixed 'result' column
import time

response = requests.post(f"{NOETL_SERVER_URL}/api/run/playbook", json={"path": TEST_PATH})

if response.status_code == 200:
    test_exec_id = response.json()['execution_id']
    print(f"✓ Test execution: {test_exec_id}\n")
    
    # Wait and poll for iterator_started + iteration jobs
    for i in range(15):
        time.sleep(2)
        
        with psycopg.connect(**{k: v for k, v in DB_CONFIG.items()}) as conn:
            with conn.cursor() as cur:
                # Check for iterator_started
                cur.execute("SELECT 1 FROM noetl.event WHERE execution_id = %s AND event_type = 'iterator_started'", (test_exec_id,))
                if cur.fetchone():
                    print(f"✓ iterator_started emitted ({(i+1)*2}s)")
                    
                    time.sleep(1)
                    
                    # Check for iteration jobs
                    cur.execute("SELECT execution_id, node_name FROM noetl.queue WHERE parent_execution_id = %s", (test_exec_id,))
                    jobs = cur.fetchall()
                    
                    if jobs:
                        print(f"\n🎉🎉🎉 PHASE 2 SUCCESS! 🎉🎉🎉\n")
                        print(f"Found {len(jobs)} iteration jobs enqueued:")
                        for job_id, node in jobs:
                            print(f"  • execution_id={job_id}, node={node}")
                        print("\n✅ Server orchestration working!")
                        print("✅ Iteration jobs properly enqueued!")
                        print("✅ ALL cursor bugs fixed!")
                        break
                    else:
                        print("⚠ No iteration jobs yet, checking logs...")
                        result = subprocess.run(
                            ['kubectl', 'logs', '-n', 'noetl', '-l', 'app=noetl-server', '--tail=30'],
                            capture_output=True, text=True
                        )
                        for line in result.stdout.split('\n'):
                            if 'ERROR' in line or str(test_exec_id) in line:
                                print(f"  {line}")
                        break
                        
        print(f"Waiting... ({(i+1)*2}s)")
else:
    print(f"✗ Failed: {response.status_code} - {response.text}")

✓ Test execution: 511697834110877954

Waiting... (2s)
✓ iterator_started emitted (4s)
⚠ No iteration jobs yet, checking logs...
                 "execution_id": "511697834110877954",
       Message: Step started event emission TODO - execution_id=511697834110877954
       Message: Step 'validate_results' failed in execution 511697834110877954: Expected 2 endpoint iterations, got 1
       Message: TODO: Emit step_failed and playbook_failed events for execution 511697834110877954


In [30]:
# Check what event type has iterator_started in result
import polars as pl

with psycopg.connect(**{k: v for k, v in DB_CONFIG.items()}) as conn:
    query = """
    SELECT event_id, execution_id, event_type, node_type, status, result::text
    FROM noetl.event
    WHERE execution_id = 511687636918993120
    AND result::text LIKE '%iterator_started%true%'
    ORDER BY created_at
    """
    events_df = pl.read_database(query, connection=conn)
    
print(f"Found {len(events_df)} events with iterator_started flag:")
print(events_df)

Found 1 events with iterator_started flag:
shape: (1, 6)
┌───────────────────────┬──────────────────────┬─────────────┬───────────┬───────────┬─────────────┐
│ event_id              ┆ execution_id         ┆ event_type  ┆ node_type ┆ status    ┆ result      │
│ ---                   ┆ ---                  ┆ ---         ┆ ---       ┆ ---       ┆ ---         │
│ i64                   ┆ i64                  ┆ str         ┆ str       ┆ str       ┆ str         │
╞═══════════════════════╪══════════════════════╪═════════════╪═══════════╪═══════════╪═════════════╡
│ 511687671522001131    ┆ 511687636918993120   ┆ step_result ┆ iterator  ┆ COMPLETED ┆ {"message": │
│                       ┆                      ┆             ┆           ┆           ┆ "Iterator   │
│                       ┆                      ┆             ┆           ┆           ┆ analysis…   │
└───────────────────────┴──────────────────────┴─────────────┴───────────┴───────────┴─────────────┘


In [32]:
# Check event details for iterator step_result
import json

with psycopg.connect(**{k: v for k, v in DB_CONFIG.items()}) as conn:
    with conn.cursor(row_factory=psycopg.rows.dict_row) as cur:
        cur.execute("""
            SELECT event_id, event_type, node_type, node_name, result, context
            FROM noetl.event
            WHERE execution_id = 511687636918993120
            AND event_type = 'step_result'
            AND node_type = 'iterator'
            ORDER BY created_at
        """)
        row = cur.fetchone()
        
if row:
    print(f"event_id: {row['event_id']}")
    print(f"event_type: {row['event_type']}")
    print(f"node_type: {row['node_type']}")
    print(f"node_name: {row['node_name']}")
    print(f"\nresult: {json.dumps(row['result'], indent=2)}")
    print(f"\ncontext: {json.dumps(row['context'], indent=2) if row['context'] else 'None'}")

event_id: 511687671522001131
event_type: step_result
node_type: iterator
node_name: fetch_all_endpoints

result: {
  "message": "Iterator analysis complete, server will execute iterations",
  "iterator_started": true,
  "total_iterations": 2
}

context: None


In [34]:
# Check ALL events for latest test execution to see full event flow
exec_id = 511691852421005553

with psycopg.connect(**{k: v for k, v in DB_CONFIG.items()}) as conn:
    with conn.cursor(row_factory=psycopg.rows.dict_row) as cur:
        cur.execute("""
            SELECT event_id, event_type, node_type, node_name, status, 
                   CASE WHEN result IS NOT NULL THEN jsonb_pretty(result::jsonb) ELSE 'NULL' END as result_preview
            FROM noetl.event
            WHERE execution_id = %s
            ORDER BY created_at
        """, (exec_id,))
        events = cur.fetchall()

print(f"Total events for execution {exec_id}: {len(events)}\n")
for i, evt in enumerate(events, 1):
    print(f"{i}. event_type={evt['event_type']}, node_type={evt['node_type']}, node_name={evt['node_name']}, status={evt['status']}")
    if 'iterator' in str(evt['node_type']) or 'loop' in str(evt['result_preview']).lower():
        print(f"   Result preview: {evt['result_preview'][:200]}")
    print()

Total events for execution 511691852421005553: 14

1. event_type=playbook_started, node_type=execution, node_name=tests/pagination/loop_with_pagination/loop_with_pagination, status=STARTED

2. event_type=workflow_initialized, node_type=workflow, node_name=workflow, status=COMPLETED

3. event_type=step_started, node_type=http, node_name=fetch_all_endpoints, status=RUNNING

4. event_type=action_started, node_type=iterator, node_name=fetch_all_endpoints, status=RUNNING
   Result preview: NULL

5. event_type=iterator_started, node_type=iterator, node_name=iterator, status=RUNNING
   Result preview: NULL

6. event_type=action_completed, node_type=iterator, node_name=fetch_all_endpoints, status=COMPLETED
   Result preview: NULL

7. event_type=step_completed, node_type=http, node_name=fetch_all_endpoints, status=COMPLETED

8. event_type=step_started, node_type=python, node_name=validate_results, status=RUNNING

9. event_type=step_result, node_type=iterator, node_name=fetch_all_endpoints, stat

In [35]:
# Check iterator_started event details (event #5)
with psycopg.connect(**{k: v for k, v in DB_CONFIG.items()}) as conn:
    with conn.cursor(row_factory=psycopg.rows.dict_row) as cur:
        cur.execute("""
            SELECT event_id, event_type, node_type, result, context
            FROM noetl.event
            WHERE execution_id = %s
            AND event_type = 'iterator_started'
            ORDER BY created_at
        """, (exec_id,))
        evt = cur.fetchone()

if evt:
    print(f"iterator_started event found!")
    print(f"event_id: {evt['event_id']}")
    print(f"result: {evt['result']}")
    print(f"context: {json.dumps(evt['context'], indent=2) if evt['context'] else 'NULL'}")

iterator_started event found!
event_id: 511691889599316215
result: None
context: {
  "mode": "sequential",
  "enumerate": false,
  "chunk_size": null,
  "collection": [
    {
      "name": "assessments",
      "path": "/api/v1/assessments",
      "page_size": 10
    },
    {
      "name": "users",
      "path": "/api/v1/users",
      "page_size": 15
    }
  ],
  "concurrency": 1,
  "nested_task": {
    "url": "{{ workload.api_url }}{{ endpoint.path }}",
    "args": {},
    "name": null,
    "sink": null,
    "tool": "http",
    "retry": {
      "on_success": {
        "while": "{{ response.paging.hasMore == true }}",
        "collect": {
          "into": "pages",
          "path": "data",
          "strategy": "append"
        },
        "next_call": {
          "params": {
            "page": "{{ (response.paging.page | int) + 1 }}",
            "pageSize": "{{ response.paging.pageSize }}"
          }
        },
        "max_attempts": 10
      }
    },
    "method": "GET",
    "para

In [38]:
# Check queue table for iteration jobs
exec_id = 511697834110877954

queue_query = f"""
SELECT execution_id, node_name, parent_execution_id, catalog_id, 
       context->>'iteration_index' as iteration_index,
       context->>'iterator_name' as iterator_name,
       created_at
FROM noetl.queue 
WHERE parent_execution_id = {exec_id}
ORDER BY created_at
"""

df = query_to_polars(queue_query)
print(f"Queue jobs for execution {exec_id}:")
for row in df.iter_rows(named=True):
    print(f"  Job: execution={row['execution_id']}, node={row['node_name']}, "
          f"iteration_index={row['iteration_index']}, iterator={row['iterator_name']}, "
          f"created={row['created_at']}")
print(f"\nTotal iteration jobs: {len(df)}")

Queue jobs for execution 511697834110877954:

Total iteration jobs: 0


In [39]:
# Check events for this execution
exec_id = 511697834110877954

events_query = f"""
SELECT event_id, event_type, node_type, node_name, 
       result IS NOT NULL as has_result,
       context IS NOT NULL as has_context
FROM noetl.event 
WHERE execution_id = {exec_id}
ORDER BY event_id
"""

df = query_to_polars(events_query)
print(f"Events for execution {exec_id}:")
for idx, row in enumerate(df.iter_rows(named=True), 1):
    print(f"  {idx}. {row['event_type']}: node_type={row['node_type']}, "
          f"node={row['node_name']}, result={row['has_result']}, context={row['has_context']}")
print(f"\nTotal events: {len(df)}")

Events for execution 511697834110877954:
  1. playbook_started: node_type=execution, node=tests/pagination/loop_with_pagination/loop_with_pagination, result=False, context=True
  2. workflow_initialized: node_type=workflow, node=workflow, result=False, context=True
  3. step_started: node_type=http, node=fetch_all_endpoints, result=False, context=True
  4. action_started: node_type=iterator, node=fetch_all_endpoints, result=False, context=True
  5. iterator_started: node_type=iterator, node=iterator, result=False, context=True
  6. action_completed: node_type=iterator, node=fetch_all_endpoints, result=False, context=True
  7. step_completed: node_type=http, node=fetch_all_endpoints, result=False, context=False
  8. step_started: node_type=python, node=validate_results, result=False, context=True
  9. step_result: node_type=iterator, node=fetch_all_endpoints, result=True, context=False
  10. action_started: node_type=task, node=validate_results, result=False, context=True
  11. action_f

In [40]:
# Test Phase 2 fix - orchestrator should wait for iterations to complete
print("🧪 Testing Phase 2: Iterator job enqueueing with pending check")
print("="*60)

# Start new test execution
test_response = start_test()
exec_id = test_response['execution_id']
print(f"✓ Test execution: {exec_id}")

# Wait for iterator_started event
time.sleep(2)
events_query = f"SELECT COUNT(*) as count FROM noetl.event WHERE execution_id = {exec_id} AND event_type = 'iterator_started'"
df = query_to_polars(events_query)
if df['count'][0] > 0:
    print(f"✓ iterator_started emitted ({time.time() - test_response['start_time']:.0f}s)")
else:
    print(f"⚠ No iterator_started yet, waiting...")

# Check for iteration jobs in queue
time.sleep(2)
queue_query = f"""
SELECT execution_id, node_name, status,
       context->>'iteration_index' as iteration_index,
       context->>'iterator_name' as iterator_name
FROM noetl.queue 
WHERE parent_execution_id = {exec_id}
ORDER BY created_at
"""
df = query_to_polars(queue_query)
if len(df) > 0:
    print(f"✓ {len(df)} iteration jobs enqueued!")
    for row in df.iter_rows(named=True):
        print(f"  - Iteration {row['iteration_index']}: {row['iterator_name']}, status={row['status']}")
else:
    print("✗ No iteration jobs found!")

# Check if validate_results was prematurely enqueued
premature_query = f"""
SELECT node_name, status FROM noetl.queue 
WHERE execution_id = {exec_id} AND node_name = 'validate_results'
"""
df2 = query_to_polars(premature_query)
if len(df2) > 0:
    print(f"✗ FAIL: validate_results was enqueued prematurely (before iterations complete)!")
else:
    print("✓ PASS: validate_results NOT enqueued yet (waiting for iterations)")

print("="*60)

🧪 Testing Phase 2: Iterator job enqueueing with pending check
Starting test: tests/pagination/loop_with_pagination/loop_with_pagination
✓ Test started
  Execution ID: 511703850881909011
  Status: running
✓ Test execution: 511703850881909011
⚠ No iterator_started yet, waiting...
✗ No iteration jobs found!
✓ PASS: validate_results NOT enqueued yet (waiting for iterations)


In [41]:
# Check if iterator_started event was emitted for latest test
exec_id = 511703850881909011

events_query = f"""
SELECT event_id, event_type, node_type, node_name, created_at
FROM noetl.event 
WHERE execution_id = {exec_id}
ORDER BY created_at
"""

df = query_to_polars(events_query)
print(f"Events for execution {exec_id}:")
for idx, row in enumerate(df.iter_rows(named=True), 1):
    print(f"  {idx}. {row['event_type']}: node_type={row['node_type']}, node={row['node_name']}, time={row['created_at']}")
print(f"\nTotal events: {len(df)}")

Events for execution 511703850881909011:
  1. playbook_started: node_type=execution, node=tests/pagination/loop_with_pagination/loop_with_pagination, time=2025-12-07 00:24:12.536006
  2. workflow_initialized: node_type=workflow, node=workflow, time=2025-12-07 00:24:12.547591
  3. step_started: node_type=http, node=fetch_all_endpoints, time=2025-12-07 00:24:12.559650
  4. action_started: node_type=iterator, node=fetch_all_endpoints, time=2025-12-07 00:24:16.911846
  5. iterator_started: node_type=iterator, node=iterator, time=2025-12-07 00:24:16.936297
  6. action_completed: node_type=iterator, node=fetch_all_endpoints, time=2025-12-07 00:24:16.951445
  7. step_completed: node_type=http, node=fetch_all_endpoints, time=2025-12-07 00:24:16.963362
  8. step_started: node_type=python, node=validate_results, time=2025-12-07 00:24:16.968061
  9. step_result: node_type=iterator, node=fetch_all_endpoints, time=2025-12-07 00:24:16.980677
  10. action_started: node_type=task, node=validate_result

In [46]:
# ✅ FINAL TEST: Pure server-side loop execution
print("🎯 FINAL TEST: Pure server-side loop execution (Phase 2)")
print("="*70)

test_response = start_test()
exec_id = test_response['execution_id']
print(f"✓ Test execution: {exec_id}\n")

# Wait for orchestrator to process
time.sleep(5)

# Check events
events_query = f"""
SELECT event_type, node_type, node_name
FROM noetl.event 
WHERE execution_id = {exec_id}
ORDER BY created_at
"""
df_events = query_to_polars(events_query)
print(f"📋 Events: {len(df_events)}")
for idx, row in enumerate(df_events.iter_rows(named=True), 1):
    marker = "🔄" if "iterator" in row['event_type'] else "  "
    print(f"{marker} {idx}. {row['event_type']}: {row['node_type']}/{row['node_name']}")

# Check iteration jobs
queue_query = f"""
SELECT execution_id, node_name, status,
       context->>'iteration_index' as idx,
       context->>'iterator_name' as iter
FROM noetl.queue 
WHERE parent_execution_id = {exec_id}
ORDER BY created_at
"""
df_queue = query_to_polars(queue_query)
print(f"\n📦 Iteration Jobs: {len(df_queue)}")
for row in df_queue.iter_rows(named=True):
    print(f"   ✓ #{row['idx']}: {row['iter']}, status={row['status']}")

# Check premature next step
premature_query = f"SELECT 1 FROM noetl.queue WHERE execution_id = {exec_id} AND node_name = 'validate_results'"
df_premature = query_to_polars(premature_query)

print(f"\n🎯 Result:")
if len(df_queue) == 2:
    print("   ✅ SUCCESS: 2 iteration jobs enqueued!")
else:
    print(f"   ❌ FAIL: Expected 2 jobs, got {len(df_queue)}")

if len(df_premature) == 0:
    print("   ✅ PASS: Next step not enqueued prematurely")
else:
    print("   ❌ FAIL: Next step enqueued before iterations complete")

print("="*70)

🎯 FINAL TEST: Pure server-side loop execution (Phase 2)
Starting test: tests/pagination/loop_with_pagination/loop_with_pagination


HTTPError: 400 Client Error: Bad Request for url: http://localhost:8082/api/run/playbook